In [9]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# --- Load and preprocess data ---
data = pd.read_csv("datatest_3.csv")
data['Date'] = pd.to_datetime(data['Date'], format="%b-%Y")
data.set_index('Date', inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')
data.dropna(inplace=True)

# --- Select columns ---
#endog_cols = [
#    'LNG 174K CBM (2-stroke dual fuel) Spot Rate\n(avg., $/day)',
#    'LNG Capacity (CBM)',
#    'Spread\nEU-US', 'Spread\nASIA-US', 'Spread\nASIA-EU'
#]
#exog_cols = [
#    'Global price of Natural Gas, Asia (start month, MMBTU)',
#    'Global price of Natural Gas, EU (start month, MMBTU)',
#    'Global price of Natural Gas, US Henry Hub (start month, MMBTU)'
#]

endog_cols = [
    'LNG 174K CBM (2-stroke dual fuel) Spot Rate\n(avg., $/day)',
    'LNG Capacity (CBM)',
    'Spread\nEU-US', 
    'Spread\nASIA-US'
]
exog_cols = [
    'Global price of Natural Gas, Asia (start month, MMBTU)',
    'Global price of Natural Gas, EU (start month, MMBTU)'
]

endog = data[endog_cols].copy()
exog = data[exog_cols].copy()

# --- Make data stationary ---
for df in [endog, exog]:
    for col in df.columns:
        if adfuller(df[col])[1] > 0.05:
            df[col] = df[col].diff()
    df.dropna(inplace=True)

# Align datasets to ensure matching dates
endog, exog = endog.align(exog, join='inner', axis=0)

# --- Scale data ---
endog_scaler = StandardScaler()
exog_scaler = StandardScaler()

endog_scaled = pd.DataFrame(endog_scaler.fit_transform(endog), index=endog.index, columns=endog.columns)
exog_scaled = pd.DataFrame(exog_scaler.fit_transform(exog), index=exog.index, columns=exog.columns)

# --- Train-test split ---
train_size = int(len(endog_scaled) * 0.9)
endog_train = endog_scaled.iloc[:train_size]
endog_test = endog_scaled.iloc[train_size:]
exog_train = exog_scaled.iloc[:train_size]
exog_test = exog_scaled.iloc[train_size:]

# --- Fit VARMAX model ---
model = VARMAX(endog_train, exog=exog_train, order=(1, 0))
model_fit = model.fit(disp=False)
print(f"✅ VARMAX model fitted with order (5, 0)")

# --- Forecast ---
forecast_scaled = model_fit.forecast(steps=len(exog_test), exog=exog_test)
forecast = pd.DataFrame(endog_scaler.inverse_transform(forecast_scaled),
                        index=endog_test.index, columns=endog.columns)

# --- Actual test data (inverse scaled) ---
endog_test_original = pd.DataFrame(endog_scaler.inverse_transform(endog_test),
                                   index=endog_test.index, columns=endog.columns)

# --- Evaluate forecast ---
print("\n📊 Mean Squared Errors:")
for col in endog.columns:
    mse = mean_squared_error(endog_test_original[col], forecast[col])
    print(f"{col}: {mse:.2f}")

# --- Plot Forecast vs Actual for main variable ---
target_col = 'LNG 174K CBM (2-stroke dual fuel) Spot Rate\n(avg., $/day)'

plt.figure(figsize=(14, 6))
plt.plot(endog_test_original.index, endog_test_original[target_col], label="Actual", linestyle='--')
plt.plot(forecast.index, forecast[target_col], label="Forecast", linestyle='-')
plt.title(f"Forecast vs Actual - {target_col}")
plt.xlabel("Date")
plt.ylabel("Spot Rate ($/day)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/statespace/varmax.py:373: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


LinAlgError: Schur decomposition solver error.